<a href="https://colab.research.google.com/github/JuanJoseMV/neuraltextgen/blob/main/RNN_TextGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%capture
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
# ! unzip '/content/wiki-news-300d-1M.vec.zip'
# import gensim.models.wrappers.fasttext
# model = gensim.models.KeyedVectors.load_word2vec_format('/content/wiki-news-300d-1M.vec')
# word_vectors = model.wv

# weights = torch.FloatTensor(word_vectors.vectors)
# embedding = nn.Embedding.from_pretrained(weights)

**Imports and commands**

In [2]:
%%capture
# ! pip install transformers
! git clone https://github.com/JuanJoseMV/neuraltextgen.git

import torch
import torch.nn as nn
import numpy as np
# from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModel, BertConfig, AutoConfig

**Cleaning the dataset**


In [ ]:
with open('/content/wiki.train.tokens') as f:
  content = f.readlines()

In [ ]:
clean = []
for c in content:
  clean.append(c.replace('\n', '<eos>'))

**Network**



> Test network






In [ ]:
dropout = 0.5

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()        
    self.embedding = nn.Embedding.from_pretrained(weights)
    self.lstm = nn.LSTM(weights.shape[1], weights.shape[1], bidirectional=True, dropout=dropout)
    self.fc1 = nn.Linear(300, 300)  
    self.softmax = nn.Softmax(dim=1)
    self.output = nn.Linear(300, word_len)
  
  def forward(self, sentence, previous_state):        
    embeds = self.embedding(torch.LongTensor([word_vectors.vocab[sentence].index]))
    lstm_out, state = self.lstm(embeds, previous_state)
    # lstm_out = self.fc1(lstm_out)
    # lstm_out = self.output(lstm_out)
    # lstm_out = self.softmax(lstm_out)
    return lstm_out, state
    
# input_layer = torch.rand(10)
# net = Net()
# result = net(input_layer)




> Implemented RNN with wiki



In [3]:
'''
Code taken from https://github.com/ChunML/NLP/blob/32a52dc6a252175c60b44389a020fda17a6339b7/text_generation/train_pt.py#L24
Blog: https://trungtran.io/2019/02/08/text-generation-with-pytorch/
'''

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace


flags = Namespace(
    train_file='/content/neuraltextgen/data/wiki103.5k.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['I', 'am'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
)


def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text = text.split()
    text = text[:int(len(text) * 0.1)]

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text


def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]


class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.lstm = nn.LSTM(lstm_size,
                            lstm_size,
                            batch_first=True)
        self.lstm = nn.LSTM(lstm_size,
                            lstm_size,
                            batch_first=True)
        self.lstm = nn.LSTM(lstm_size,
                            lstm_size,
                            batch_first=True)                        
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))


def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer


def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()
    words = ['the', 'only']

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])

    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words).encode('utf-8'))


def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0

    for e in range(200):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            net.train()

            optimizer.zero_grad()

            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            loss_value = loss.item()

            loss.backward()

            state_h = state_h.detach()
            state_c = state_c.detach()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), flags.gradients_norm)

            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                # torch.save(net.state_dict(),
                #            'checkpoint_pt/model-{}.pth'.format(iteration))


if __name__ == '__main__':
    main()                  

Vocabulary size 3775
Epoch: 4/200 Iteration: 100 Loss: 5.347992420196533
Epoch: 8/200 Iteration: 200 Loss: 3.4410104751586914
Epoch: 13/200 Iteration: 300 Loss: 2.2754533290863037
Epoch: 17/200 Iteration: 400 Loss: 1.8583910465240479
Epoch: 21/200 Iteration: 500 Loss: 1.43454909324646
Epoch: 26/200 Iteration: 600 Loss: 0.9589431285858154
Epoch: 30/200 Iteration: 700 Loss: 0.8182269334793091
Epoch: 34/200 Iteration: 800 Loss: 0.6255089044570923
Epoch: 39/200 Iteration: 900 Loss: 0.4264347553253174
Epoch: 43/200 Iteration: 1000 Loss: 0.3103407025337219
b'the only work incorporating two but unarmored at 0600 UTC on death were their respective to develop . Through the United colossal guns and attack . Upon a small @-@ back in an email , having their progress UB @-@ steel twin of all of the 4th , were captured by twenty further and developed one in " bar their own slightly been at by Matthew 5 @,@ 540 through communicate their battlefleet had the reputation it to have appeared in the countr

> Implemented RNN with tbc

In [4]:
'''
Code taken from https://github.com/ChunML/NLP/blob/32a52dc6a252175c60b44389a020fda17a6339b7/text_generation/train_pt.py#L24
Blog: https://trungtran.io/2019/02/08/text-generation-with-pytorch/
'''

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace


flags = Namespace(
    train_file='/content/neuraltextgen/data/tbc.5k.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['the', 'only'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
)


def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text = text.split()
    text = text[:int(len(text) * 0.1)]

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text


def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]


class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))


def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer


def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()
    # words = ['the', 'only']

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])

    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words).encode('utf-8'))


def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0

    for e in range(200):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            net.train()

            optimizer.zero_grad()

            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            loss_value = loss.item()

            loss.backward()

            state_h = state_h.detach()
            state_c = state_c.detach()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), flags.gradients_norm)

            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
            #     torch.save(net.state_dict(),
            #                'checkpoint_pt/model-{}.pth'.format(iteration))


if __name__ == '__main__':
    main()            

Vocabulary size 1859
[15, 299, 83, 300, 0, 2, 538, 301, 539, 2, 540, 541, 1, 302, 25, 303, 1, 542, 55, 9, 304, 543, 305, 16, 170, 171, 544, 4, 306, 0, 208, 21, 48, 7, 43, 209, 1, 545, 0, 546, 101, 1, 12, 547, 9, 548, 0, 6, 549, 52, 8, 44, 5, 307, 550, 1, 37, 49, 308, 28, 309, 210, 4, 551, 552, 29, 2, 553, 7, 554, 16, 310, 1, 4, 555, 556, 2, 557, 0, 2, 172, 311, 0, 558, 211, 48, 7, 14, 173, 1, 559, 25, 560, 0, 561, 116, 1, 23, 14, 75, 49, 26, 2, 562, 0, 15, 173, 49, 312, 55, 15, 143, 4, 15, 313, 563, 39, 1, 59, 10, 45, 53, 564, 31, 565, 0, 16, 212, 213, 42, 38, 566, 67, 3, 102, 2, 567, 314, 0, 10, 568, 144, 9, 315, 7, 569, 570, 1, 68, 571, 69, 9, 145, 214, 572, 32, 573, 574, 0, 575, 576, 0, 84, 316, 215, 577, 1, 60, 317, 2, 96, 0, 578, 146, 80, 216, 1, 5, 318, 1, 23, 2, 579, 580, 85, 581, 582, 583, 2, 584, 7, 42, 86, 16, 585, 48, 3, 586, 587, 588, 4, 3, 589, 2, 590, 3, 2, 591, 319, 592, 217, 2, 593, 7, 594, 7, 595, 319, 596, 597, 1, 4, 37, 598, 4, 320, 17, 87, 214, 321, 4, 599, 322, 600

KeyboardInterrupt: ignored



> LSTM o top of pretrained model



/bin/bash: nano: command not found
